In [2]:
# # possibly try using multiprocessing (as of now didn't work)
# # https://www.youtube.com/results?search_query=concurrency+python
# # https://www.realpythonproject.com/how-to-parallelize-for-loops-in-python-and-work-with-shared-dictionaries/

# def generate_false_samples(entities_dict, sample, head_or_tail, training_data, index_start_of_relation, allow_shift):
#     '''
    
#     '''
# #     running = True
#     counter = 0
#     head_or_tail = random.randint(0, 1)

#     # 1 for head
#     if head_or_tail:
#         # generate false head entity
#         # to break in case our tail entity has a relation with every other entity 
#         while(True):
#             unique = True
#             random_entity = random.choice(entities_dict)
#             for i in range(index_start_of_relation, len(training_data[index_start_of_relation:])):
#                 if random_entity == training_data[i][0] and sample[2] == training_data[i][2]:
#                     unique = False
#                     running = False
#                     break
#                 if training_data[i][1] != training_data[index_start_of_relation][1]:
#                     running = False
#                     break
                    
#             counter += 1
#             if unique or counter == 100:
#                 break
#         if allow_shift and not unique:
#             sample = generate_false_samples(entities_dict, sample, 0, training_data, index_start_of_relation, False)

#         elif unique:
#             sample.append(random_entity)
#             sample.append(sample[2])
#         else:
#             print("mistake")
            
#     # 0 for tail
#     else:
#         # generate false tail entity
#         while(True):
#             random_entity = random.choice(entities_dict)
#             unique = True
#             for i in range(index_start_of_relation, len(training_data[index_start_of_relation:])):
#                 if sample[0] == training_data[i][0] and random_entity == training_data[i][2]:
#                     unique = False
#                     running = False
#                     break
                    
#                 if training_data[i][1] != training_data[index_start_of_relation][1]:
#                     running = False
#                     break
#             counter += 1
#             if unique or counter == 100:
#                 break
        
#         if allow_shift and not unique:
#             sample = generate_false_samples(entities_dict, sample, 1, training_data, index_start_of_relation, False) 
#         elif unique:
#             sample.append(sample[0]) 
#             sample.append(random_entity)
#         else:
#             print("mistake")
#     return sample

In [1]:
import math
import torch
import pandas as pd
from operator import itemgetter
import random
from pathlib import Path
from torch.utils.data import Dataset, DataLoader

### VIP: have to test the model with more negative samples (as of now we generate only one -ve sample per +ve)

In [2]:
# also maybe use named_tuples to represent the (h,r,t,h',t')

# if torch.cuda.is_available():  
#   dev = "cuda:0" 
# else:  
#   dev = "cpu"
# convert code to use cuda in the end 
# https://medium.com/ai%C2%B3-theory-practice-business/use-gpu-in-your-pytorch-code-676a67faed09
# torch.cuda.is_available()

In [95]:
def generate_false_samples(entities_dict, positive_sample, training_data, index_start_of_relation, num_needed_samples):
    counter = 0
    generated_samples = []
    while len(generated_samples) < num_needed_samples and counter < 30000:
        head_or_tail = random.randint(0, 1)
        random_entity = random.choice(entities_dict)
        counter += 1
        
        # 1 for head
        if(head_or_tail):
            negative_sample = [random_entity, positive_sample[2]]    
        else:
            negative_sample = [positive_sample[0], random_entity]

        relation_is_false = is_relation_false(training_data[index_start_of_relation:], negative_sample)
        relation_already_generated = is_relation_in_samples(generated_samples, negative_sample)
            
        if is_relation_false and not(relation_already_generated):
            generated_samples.append(negative_sample)
    
    positive_sample.extend(generated_samples)
    return positive_sample
        

In [96]:
def is_relation_false(training_data, sample):
    for i in range(len(training_data)):
        unique = True
#     for i in range(index_start_of_relation, len(training_data[index_start_of_relation:])):
        if sample[0] == training_data[i][0] and sample[1] == training_data[i][2]:
            unique = False
            break
                    
        if training_data[i][1] != training_data[0][1]:
            break
    return unique

In [69]:
def is_relation_in_samples(generated_samples, sample_to_test):
    generated = False
    for sample in generated_samples:
        if sample_to_test[0] == sample[0] and sample_to_test[1] == sample[1]:
            generated = True
    return generated

In [70]:
def read_triplets_file(path, header, seperator):
    return pd.read_csv(path, header= header, sep=seperator)

In [71]:
def read_training_data(path, header, seperator):
    '''
    returns a list of sorted (according to relation) training instances 
    '''
    data = pd.read_csv(path, header=header, sep=seperator)
    training_data = data.values.tolist()
# #     print(training_data[0:10])
    training_data = sorted(training_data, key=itemgetter(1)) 
    return training_data

In [182]:
# chose the dimension to be d=500
def convert_to_dict(text_data):
    output_dict = dict()
#     torch.split(temp1, 1, dim=1)    
    matrix=torch.rand(2000, len(text_data))
    vectors = torch.split(matrix, 1, dim=1)
    
    for i in range(len(text_data)):
#         temp = torch.reshape(vectors[i][0], (-1, 1))
        output_dict[text_data[i]] = vectors[i]/torch.linalg.vector_norm(vectors[i])
        output_dict[text_data[i]].requires_grad=True
    return output_dict

In [165]:
def false_samples_generated():
    extended_data = Path("./data/wn18rr/extended_training_data.txt")
    return extended_data.is_file()

In [166]:
def encoding_training_data(entity_dict, relation_dict, training_data):
    for i in range(len(training_data)):
        for j in range(len(training_data[i])):
            if j == 1:
                training_data[i][j] = relation_dict[training_data[i][j]]
            elif j < 3:
                training_data[i][j] = entity_dict[training_data[i][j]]
            else:
                training_data[i][j] = training_data[i][j].strip("[]").split(",")
                training_data[i][j] = [entity_dict[int(training_data[i][j][0])], entity_dict[int(training_data[i][j][1])]]
            
#         training_data[i][3] = entity_dict[training_data[i][3]]
#         training_data[i][4] = entity_dict[training_data[i][4]]
    return training_data

In [167]:
def scoring_function(head, relation, tail):
    score = - torch.linalg.vector_norm(head + relation - tail)
    return score

In [168]:
# sample = (h, r, t, h', t')
def loss_func(head, relation, tail, false_samples, margin):
    
    positive_score = scoring_function(head, relation, tail)
#     print(len(false_sample))
    loss = max(0, max([margin - positive_score + scoring_function(false_head, relation, false_tail) for (false_head, false_tail) in  false_samples]))
#         loss = max(0, margin - positive_score + scoring_function(false_head, relation, false_tail)
    return loss

In [183]:
entities_temp_list = read_triplets_file('./data/wn18rr/entities.dict', header=None, seperator='\t')[1]
entities = convert_to_dict(entities_temp_list)

In [184]:
# len(entities[260881])
# entities

In [185]:
# entities_temp_list = pd.read_csv('./data/wn18rr/entities.dict', header=None, sep='\t')[1]
# entities_matrix = torch.eye(len(entities_temp_list))
# entities_vectors = torch.split(entities_matrix, 1)
# entities = dict()
# for i in range(len(entities_temp_list)):
#     entities[entities_temp_list[i]] = torch.reshape(entities_vectors[i][0], (-1, 1))
#     entities[entities_temp_list[i]].requires_grad=True

In [186]:
relation_temp_list = read_triplets_file('./data/wn18rr/relations.dict', header=None, seperator='\t')[1]
relations = convert_to_dict(relation_temp_list)
# relations

In [187]:
# len(relations)

In [188]:
# relation_temp_list = pd.read_csv('./data/wn18rr/relations.dict', header=None, sep='\t')[1]
# relation_matrix = torch.eye(len(relation_temp_list))
# relation_vectors = torch.split(relation_matrix, 1
# relations = dict()
# for i in range(len(relation_temp_list)):
#     relations[relation_temp_list[i]] = torch.reshape(relation_vectors[i][0], (-1, 1))
#     relations[relation_temp_list[i]].requires_grad=True

In [189]:
training_data = read_training_data('./data/wn18rr/train.txt', header=None, seperator='\t')
training_data[0:10]

TypeError: 'DataFrame' object is not callable

In [190]:
# oprimaly false_samples_count should be 20, so num_needed_samples= 20

if false_samples_generated():
    print("found file")
    read_training_data = pd.read_csv('./data/wn18rr/extended_training_data.txt', header=None, sep='\t')
    training_data = read_training_data.values.tolist()
else:
    for i  in range(len(training_data)):
        print(i)
        if not (training_data[i][1] == training_data[i-1][1]):
            index_start_of_relation = i
            print("new index: ", index_start_of_relation)
            print(training_data[i][1])
            
            
# def generate_false_samples(entities_dict, positive_sample, training_data, index_start_of_relation, num_needed_samples):
        training_data[i] = generate_false_samples(entities_temp_list, 
                                              training_data[i], 
                                              training_data, 
                                              index_start_of_relation,
                                              num_needed_samples=10)
    
    textfile = open("./data/wn18rr/extended_training_data.txt", "w")
    for row in training_data:
        for i in range(len(row)):
            if i != len(row)-1:
                textfile.write(str(row[i]) + "\t")
            else:
                textfile.write(str(row[i]))
        textfile.write("\n")
    textfile.close()

found file


In [191]:
# for i  in range(len(training_data)):
#     if not training_data[i][1] == training_data[i-1][1]:
#         index_start_of_relation = i
#         print("new index: ", index_start_of_relation)
#         print(training_data[i][1])
        

counter = 0
for i in training_data:
    if len(i) != 13:
        counter +=1
#         print(len(i))
print(counter)

# for sample in training_data[0:10]:
#     print(len(sample))
# training_data[2]

0


In [192]:
# we get the embeddings for (h, r, t# training_data)
training_data = encoding_training_data(entities, relations, training_data)

In [193]:
print(training_data[0][3][0])
# print(len(training_data[0][0]))
# print(len(training_data[0][1]))
# print(len(training_data[0][2]))
# print(len(training_data[0][3]))
# print(len(training_data[0][4]))
# training_data[0]

# counter = 0
# for i in training_data:
#     if len(i) != 5:
#         counter +=1
#         print(len(i))
# print(counter)

tensor([[0.0213],
        [0.0237],
        [0.0359],
        ...,
        [0.0273],
        [0.0110],
        [0.0265]], requires_grad=True)


In [194]:
# # margin = 1, dimension d = 10000

# train_loader = DataLoader(dataset=training_data,
#                           batch_size=50,
#                           shuffle=True,
#                           num_workers=0)

# # convert to an iterator and look at one random sample
# dataiter = iter(train_loader)
# data = dataiter.next()
# head, relation, tail, false_head, false_tail = data
# # print(head, "\n", relation)

# # # Dummy Training loop
# num_epochs = 5
# total_samples = len(training_data)
# n_iterations = math.ceil(total_samples/4)
# # print(total_samples, n_iterations)
# for epoch in range(num_epochs):
# #     for i, (inputs, labels) in enumerate(train_loader):
#     for i, (head, relation, tail, false_head, false_tail) in enumerate(train_loader):
        
#         # here: 178 samples, batch_size = 4, n_iters=178/4=44.5 -> 45 iterations
#         # Run your training process
# #         print(head.data)
#         # loss
#         l = loss_func(head, relation, tail, false_head, false_tail, 1)
# #         l.retain_grad()
#         head.retain_grad()
#         relation.retain_grad()
#         tail.retain_grad()
#         false_head.retain_grad() 
#         false_tail.retain_grad()
#         # calculate gradients = backward pass
#         l.backward()
        
        
#         with torch.no_grad():
#             head -= 0.1 * head.grad
#             relation -= 0.1 * relation.grad
#             tail -= 0.1 * tail.grad
#             false_head -= 0.1 * false_head.grad
#             false_tail -= 0.1 * false_tail.grad
            
#             # don't forget to zero the gradients
       
        
#         head.grad.zero_()
#         relation.grad.zero_()
#         tail.grad.zero_()
#         false_head.grad.zero_() 
#         false_tail.grad.zero_() 
        
#         if i%100 == 0:
#             print(f'Iteration: {i} | Loss {l.item()}')










# results: 
# Iteration: 0 | Loss 1.0015830993652344
# Iteration: 100 | Loss 0.9973239898681641
# Iteration: 200 | Loss 0.9993381500244141
# Iteration: 300 | Loss 0.9902715682983398
# Iteration: 400 | Loss 1.0056066513061523
# Iteration: 500 | Loss 0.9979467391967773
# Iteration: 600 | Loss 1.0004215240478516
# Iteration: 700 | Loss 0.9965410232543945

In [195]:
# margin = 1, dimension d = 20000
train_loader = DataLoader(dataset=training_data,
                          batch_size=50,
                          shuffle=True,
                          num_workers=0)

# convert to an iterator and look at one random sample
dataiter = iter(train_loader)
data = dataiter.next()
head = data[0]
relation = data[1]
tail = data[2]
false_samples = data[3:]
# print(head, "\n", relation)

# # Dummy Training loop
num_epochs = 7
total_samples = len(training_data)
n_iterations = math.ceil(total_samples/4)
# print(total_samples, n_iterations)
for epoch in range(num_epochs):
#     for i, (inputs, labels) in enumerate(train_loader):
    for i, data in enumerate(train_loader):
        head = data[0]
        relation = data[1]
        tail = data[2]
        false_samples = data[3:]
        # here: 178 samples, batch_size = 4, n_iters=178/4=44.5 -> 45 iterations
        # Run your training process
#         print(head.data)
        # loss
        l = loss_func(head, relation, tail, false_samples, margin=0.1)
#         l.retain_grad()
        head.retain_grad()
        relation.retain_grad()
        tail.retain_grad()
#         print(len(false_samples))
#         false_samples.retain_grad() 
        # calculate gradients = backward pass
        l.backward()
        
        
        with torch.no_grad():
            head -= 0.001 * head.grad
            relation -= 0.001 * relation.grad
            tail -= 0.001 * tail.grad
#             false_head -= 0.1 * false_head.grad
#             false_tail -= 0.1 * false_tail.grad
            
            # don't forget to zero the gradients
       
        
        head.grad.zero_()
        relation.grad.zero_()
        tail.grad.zero_()
#         false_head.grad.zero_() 
#         false_tail.grad.zero_() 
        
        if i%100 == 0:
            print(f'Iteration: {i} | Loss {l.item()}')


Iteration: 0 | Loss 0.11751079559326172
Iteration: 100 | Loss 0.11075687408447266
Iteration: 200 | Loss 0.11855792999267578
Iteration: 300 | Loss 0.1144418716430664
Iteration: 400 | Loss 0.11540889739990234
Iteration: 500 | Loss 0.11643600463867188
Iteration: 600 | Loss 0.1112203598022461
Iteration: 700 | Loss 0.12097930908203125
Iteration: 800 | Loss 0.10689640045166016
Iteration: 900 | Loss 0.11993789672851562
Iteration: 1000 | Loss 0.1267843246459961
Iteration: 1100 | Loss 0.09723949432373047
Iteration: 1200 | Loss 0.11523151397705078
Iteration: 1300 | Loss 0.1022653579711914
Iteration: 1400 | Loss 0.10685062408447266
Iteration: 1500 | Loss 0.11931800842285156
Iteration: 1600 | Loss 0.11276626586914062
Iteration: 1700 | Loss 0.12514877319335938
Iteration: 0 | Loss 0.11346244812011719
Iteration: 100 | Loss 0.1211404800415039
Iteration: 200 | Loss 0.11661529541015625
Iteration: 300 | Loss 0.11554622650146484
Iteration: 400 | Loss 0.11117744445800781
Iteration: 500 | Loss 0.10934638977

KeyboardInterrupt: 